In [16]:
import pandas as pd

df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Cleaned_Code\3_5.csv')

df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,address,year,major_city,state,chain,address_standardized_ON_parenthesis,address_standardized_OFF_parenthesis,Address_Type
0,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 I-90 ( MATP ) MM 29 EB,<U+25A1> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,I-90 ( MATP ) MM 29 EB,2006,Blandford,MA,Exxon,( MATP ) MM 29 EB,I-90,empty
1,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 I-90 ( MATP ) MM 29 EB,24 HRS S,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,I-90 ( MATP ) MM 29 EB,2007,Blandford,MA,Exxon,( MATP ) MM 29 EB,I-90,empty
2,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 I-90 ( MATP ) MM 29 EB,HAS 24 SO <U+2610> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,I-90 ( MATP ) MM 29 EB,2008,Blandford,MA,Exxon,( MATP ) MM 29 EB,I-90,empty
3,D Blandford ( 01008 ) Blandford Plaza EB # 902...,413-848-2056 I-90 ( MATP ) MM 29 EB,24 S <U+2610>,Blandford,1008,Blandford Plaza EB # 9020 ( Gulf ),413-848-2056,I-90 ( MATP ) MM 29 EB,2014,Blandford,MA,Gulf,( MATP ) MM 29 EB,I-90,empty
4,D Blandford ( 01008 ) Blandford Plaza EB # 902...,2 413-848-2056 I-90 ( MATP ) MM 29 EB,24 S,Blandford,1008,Blandford Plaza EB # 9020 ( Gulf ),413-848-2056,I-90 ( MATP ) MM 29 EB,2015,Blandford,MA,Gulf,( MATP ) MM 29 EB,I-90,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38130,"C Tok , 99780 Village Gas",6 907-883-4660 AK 1 ( MM 1313.2 ),NaN,Tok,99780,Village Gas,907-883-4660,AK 1 ( MM 1313.2 ),2007,Tok,AK,NaN,( MM 1313.2 ),AK 1,empty
38131,"C Tok , 99780 Village Gas",6 907-883-4660 AK 1 ( MM 1313.2 ),<U+25C9>,Tok,99780,Village Gas,907-883-4660,AK 1 ( MM 1313.2 ),2008,Tok,AK,NaN,( MM 1313.2 ),AK 1,empty
38132,"Tok , 99780 Plaza Truck Stop ( Texaco )",907-883-5833 AK Hwy 2 ( MM 1313.5 ),NaN,Tok,99780,Plaza Truck Stop ( Texaco ),907-883-5833,AK Hwy 2 ( MM 1313.5 ),2006,Tok,AK,Texaco,( MM 1313.5 ),AK Hwy 2,empty
38133,"C Tok , 99780 Plaza Truck Stop ( Texaco )",907-883-5833 AK Hwy 2 ( MM 1313.5 ),NaN,Tok,99780,Plaza Truck Stop ( Texaco ),907-883-5833,AK Hwy 2 ( MM 1313.5 ),2007,Tok,AK,Texaco,( MM 1313.5 ),AK Hwy 2,empty


In [17]:
#df = df[df['Flag_Reason'] == "No matches found"]
#filter for state in #filter for states CA, UT, NV, AZ
#df = df[df['state'].isin(['CA', 'UT', 'NV', 'AZ'])]
#show only columns city zip_code label phone address year major_city state chain Flagged Flag_Reason,  'address_standardized_ON_parenthesis', 'address_standardized_OFF_parenthesis'
# df = df[['city', 'zip_code', 'label', 'phone', 'address_standardized_OFF_parenthesis','address_standardized_ON_parenthesis', 
#                    'year', 'major_city', 'state', 'Flagged', 'Flag_Reason']]
#df[['address_standardized_OFF_parenthesis', 'address_standardized_ON_parenthesis', 'Address_Type','zip_code','city','label']]

In [18]:
import re

def identify_proper_addresses(df):
    """
    Function to identify proper addresses and update Address_Type column.
    Only processes rows where Address_Type is currently "empty".
    For these rows:
    1. Removes any instances of ")" and "<U+00D8>" from address_standardized_OFF_parenthesis
    2. Checks if the cleaned address starts with a number sequence (proper address)
    3. Updates Address_Type to "Proper" if criteria is met
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the address data
    
    Returns:
    pandas.DataFrame: DataFrame with updated Address_Type column and cleaned addresses
    """
    
    # Create a copy to avoid modifying the original dataframe
    df_copy = df.copy()
    
    # Ensure Address_Type column exists
    if 'Address_Type' not in df_copy.columns:
        df_copy['Address_Type'] = ''
    
    # Only iterate through rows where Address_Type is "empty"
    empty_rows = df_copy[df_copy['Address_Type'] == 'empty']
    
    for index, row in empty_rows.iterrows():
        address = row['address_standardized_OFF_parenthesis']
        
        # Check if address is not null/empty
        if pd.notna(address) and str(address).strip():
            # Convert to string and strip whitespace
            address_str = str(address).strip()
            
            # Remove any instances of ")" and "<U+00D8>" from the address
            cleaned_address = address_str.replace(')', '').replace('<U+00D8>', '')
            
            # Update the address column with the cleaned version
            df_copy.at[index, 'address_standardized_OFF_parenthesis'] = cleaned_address
            
            # Check if cleaned address starts with a number sequence followed by a space
            # This pattern matches one or more digits followed by a space
            if re.match(r'^\d+\s', cleaned_address):
                df_copy.at[index, 'Address_Type'] = 'Proper'
    
    return df_copy

# Apply the function to the dataframe
df = identify_proper_addresses(df)

# Display the results showing the relevant columns
print("Updated dataframe with Address_Type classification:")
df[['phone','address_standardized_OFF_parenthesis', 'address_standardized_ON_parenthesis', 'Address_Type','zip_code','city','label']]

Updated dataframe with Address_Type classification:


,phone,address_standardized_OFF_parenthesis,address_standardized_ON_parenthesis,Address_Type,zip_code,city,label
0,413-848-2056,I-90,( MATP ) MM 29 EB,empty,1008,Blandford,Blandford Plaza EB Exxon # 5020
1,413-848-2056,I-90,( MATP ) MM 29 EB,empty,1008,Blandford,Blandford Plaza EB Exxon # 5020
2,413-848-2056,I-90,( MATP ) MM 29 EB,empty,1008,Blandford,Blandford Plaza EB Exxon # 5020
3,413-848-2056,I-90,( MATP ) MM 29 EB,empty,1008,Blandford,Blandford Plaza EB # 9020 ( Gulf )
4,413-848-2056,I-90,( MATP ) MM 29 EB,empty,1008,Blandford,Blandford Plaza EB # 9020 ( Gulf )
...,...,...,...,...,...,...,...
38130,907-883-4660,AK 1,( MM 1313.2 ),empty,99780,Tok,Village Gas
38131,907-883-4660,AK 1,( MM 1313.2 ),empty,99780,Tok,Village Gas
38132,907-883-5833,AK Hwy 2,( MM 1313.5 ),empty,99780,Tok,Plaza Truck Stop ( Texaco )
38133,907-883-5833,AK Hwy 2,( MM 1313.5 ),empty,99780,Tok,Plaza Truck Stop ( Texaco )


In [19]:
# Function to clean "City," prefix from address entries
def remove_city_prefix(df):
    """
    Remove "City," prefix from address_standardized_OFF_parenthesis entries
    that start with this pattern.
    """
    # Create a copy to avoid modifying the original dataframe
    df_copy = df.copy()
    
    # Find entries that start with "City,"
    city_mask = df_copy['address_standardized_OFF_parenthesis'].str.startswith('City,', na=False)
    
    print(f"Found {city_mask.sum()} entries starting with 'City,'")
    
    if city_mask.sum() > 0:
        print("\nBefore cleaning:")
        print(df_copy.loc[city_mask, 'address_standardized_OFF_parenthesis'].tolist())
        
        # Remove "City," prefix (including the comma and any following space)
        df_copy.loc[city_mask, 'address_standardized_OFF_parenthesis'] = (
            df_copy.loc[city_mask, 'address_standardized_OFF_parenthesis']
            .str.replace('^City,\s*', '', regex=True)
        )
        
        print("\nAfter cleaning:")
        print(df_copy.loc[city_mask, 'address_standardized_OFF_parenthesis'].tolist())
        
        print(f"\nSuccessfully cleaned {city_mask.sum()} entries")
    
    return df_copy

# Apply the cleaning function
df = remove_city_prefix(df)

# Show the results for the affected entries
city_cleaned_entries = df[df['address_standardized_OFF_parenthesis'].str.contains('US 40 & US 189', na=False)]
if len(city_cleaned_entries) > 0:
    print("\nEntries after 'City,' removal:")
    display(city_cleaned_entries[['address_standardized_OFF_parenthesis', 'city', 'state', 'zip_code', 'Address_Type']].head())

Found 0 entries starting with 'City,'

Entries after 'City,' removal:


,address_standardized_OFF_parenthesis,city,state,zip_code,Address_Type
23202,US 40 & US 189,Heber City,UT,84032,empty
23203,"City , 84032 US 40 & US 189",NaN,UT,84032,empty
36799,US 40 & US 189,Heber City,UT,84032,empty


In [20]:
df.to_csv('3_7.csv', index=False)